In [184]:
from IPython.display import HTML
import numpy as np
import copy

shell = get_ipython()

def adjust_font_size():
    display(HTML('''<style>
        body{
            font-size: 32px;
        }
    '''))

if adjust_font_size not in shell.events.callbacks['pre_execute']:
    shell.events.register('pre_execute', adjust_font_size)

In [216]:
def printmatrix(m, pad = 4):
    # Make column labels across the top of the matrix
    for i in range(0, len(m[0]) + 1):
        print(f'{i:>{pad}}', end = '')
        
    print()
    row = 0
    for r in m:
        # Print labels for each row
        row += 1
        print(f'{str(row):>{pad}}', end = '')
        
        for d in r:
            print(f'{str(d):>{pad}}', end = '')
        print()

In [29]:
'''seq1 = "TTAGCCAGT"
seq2 = "TAAGACATTTTAC"
gappenalty = -8
scoringmatrix = [[0 for i in range(0, len(seq1) + 1)] for j in range(0, len(seq2) + 1)]
directionmatrix = [["." for i in range(0, len(seq1) + 1)] for j in range(0, len(seq2) + 1)]
print(scoringmatrix)
scoringmatrix[0][1] = 77
print(scoringmatrix)
printmatrix(scoringmatrix)'''

'seq1 = "TTAGCCAGT"\nseq2 = "TAAGACATTTTAC"\ngappenalty = -8\nscoringmatrix = [[0 for i in range(0, len(seq1) + 1)] for j in range(0, len(seq2) + 1)]\ndirectionmatrix = [["." for i in range(0, len(seq1) + 1)] for j in range(0, len(seq2) + 1)]\nprint(scoringmatrix)\nscoringmatrix[0][1] = 77\nprint(scoringmatrix)\nprintmatrix(scoringmatrix)'

In [28]:
'''

printmatrix(scoringmatrix, 4)
print(seq1)
print(seq2)
'''

'\n\nprintmatrix(scoringmatrix, 4)\nprint(seq1)\nprint(seq2)\n'

In [9]:
--TG
CATT

NameError: name 'TG' is not defined

In [12]:
printmatrix(directionmatrix, 1)

v  h  h  h  h  h  h  h  h  h  
v  ↖ ↖ ← ← ← ← ← ← ↖ 
v  ↑ ↖ ↖ ← ← ← ↖ ← ← 
v  ↑ ↖ ↖ ↖ ↖ ↖ ↖ ← ← 
v  ↑ ↖ ↑ ↖ ← ← ← ↖ ← 
v  ↑ ↖ ↖ ↑ ↖ ↖ ↖ ← ↖ 
v  ↑ ↖ ↑ ↑ ↖ ↖ ← ↖ ↖ 
v  ↑ ↖ ↖ ↑ ↑ ↖ ↖ ← ← 
v  ↖ ↖ ↑ ↑ ↑ ↖ ↑ ↖ ↖ 
v  ↖ ↖ ↖ ↑ ↑ ↖ ↑ ↖ ↖ 
v  ↖ ↖ ↖ ↑ ↑ ↖ ↑ ↖ ↖ 
v  ↖ ↖ ↖ ↑ ↑ ↖ ↑ ↖ ↖ 
v  ↑ ↑ ↖ ↑ ↑ ↖ ↖ ↖ ↑ 
v  ↑ ↑ ↑ ↖ ↖ ↖ ↑ ↖ ↑ 


In [192]:
def get_sequences():
    '''
    Returns the sequences extracted from both population files.
    '''
    
    f1 = open('pop1.txt', 'r')
    f2 = open('pop2.txt', 'r')

    p1 = get_array(f1)
    p2 = get_array(f2)
    l = np.ones_like(p1)
    
    for g, p in enumerate(p2):
        for s in range(0, len(p2[g])):
            p1[g].append(p2[g][s])
            
    l = np.insert(l, len(p2[0]), np.full((len(p2[0]), 1), 2), axis = 1)
    
    return p1, l.astype(int)

def get_array(f):
    '''
    Extracts the sequences from a population file into a two dimensional list.
    Returns list with shape = (generations, population size).
    '''
    
    pop = []
    gen = -1
    for s in f:
        if s[0] == 'P' or s[0] == '\n':
            pop.append([])
            gen += 1
            continue
        pop[gen].append(s[:-1])
    
    return pop
                      
#print(get_sequences())

In [187]:
def analyze(seq1, seq2):
    '''
    Attempt to align the two given sequences and return the scoring matrix. The result of matched characters is 
    +10, for mismatched characters is -10, and the gap penalty is -8 points.
    '''
    
    gappenalty = -8
    
    # Initialize scoring matrix
    scoringmatrix = [[0 for i in range(0, len(seq1) + 1)] for j in range(0, len(seq2) + 1)]
    
    for i in range(0, len(scoringmatrix[0])):
        scoringmatrix[0][i] =  i * gappenalty
    for i in range(0, len(scoringmatrix)):
        scoringmatrix[i][0] = i * gappenalty

    # Populate scoring matrix for given sequences
    for r in range(1, len(scoringmatrix)):
        for c in range(1, len(scoringmatrix[0])):
            # Check the score for moving in each direction
            vert = scoringmatrix[r - 1][c] + gappenalty
            horz = scoringmatrix[r][c - 1] + gappenalty
            diag = scoringmatrix[r - 1][c - 1]
            if (seq1[c - 1] == seq2[r - 1]):
                diag += 10
            else:
                diag -= 10
            # Assign the best direction score
            scoringmatrix[r][c] = max(vert, horz, diag)
            
            #print(f'Index r: {r}, Index c: {c}, Last score: {scoringmatrix[r - 1][c - 1]}')
            
    return scoringmatrix[len(scoringmatrix) - 1][len(scoringmatrix[0]) - 1]

In [198]:
def compare(gen, p):
    '''
    Attempts to align each sequence in the given generation with every other sequence and returns a table of
    each alignment score, and the median score.
    '''
    scores = np.zeros((len(p[gen]), len(p[gen]))).astype(int) # [[0 for i in range(0, len(p[gen]))] for j in range(0, len(p[gen]))]
    l = []
    
    # Check each sequence against all the other sequences in the generation
    for i, s in enumerate(p[gen]):
        for j in range(i + 1, len(p[gen])):
            #print(f'Comparing {t} to {s}')
            scores[i][j] = analyze(s, p[gen][j])
            scores[j][i] = scores[i][j] # the comparisons would be the same
            l.append(scores[i][j])
                
    return scores, np.median(np.array(l))

In [223]:
def check_row(s, med, c, key = []):
    make_key = key == []

    for y in range(0, len(s)):
        if s[c][y] == 0:
            if make_key:
                key.append(1)
            # skip the diagonal for normal checking
        elif s[c][y] > med:
            if make_key:
                key.append(1)
            else:
                s[c][y] = key[y]
        else:
            if make_key:
                key.append(2)
            else:
                s[c][y] = 3 - key[y]  

    return key
    
def predict(s, m, p):
    '''
    Predicts which group each individual belongs in, and returns an array with the final predictions.
    '''
        
    key = check_row(s, m, p)
    #print(key)
    
    for c in range(0, len(s)):
        check_row(s, m, c, key)

    #printmatrix(s, 3)
    
    return row_mode(s)

def row_mode(s):
    '''
    Returns an array of shape (len(s), ) that has the mode of each row. Assumes labels of 1 or 2.
    '''
    
    predictions = np.zeros((len(s), )).astype(int)

    for r in range(0, len(s)):
        count = 0
        for c in range(0, len(s[0])):
            if s[r][c] == 1:
                count += 1
        if count >= len(s) / 2:
            predictions[r] = 1
        else:
            predictions[r] = 2
            
    return predictions

def col_mode(s):
    '''
    Returns an array of shape (len(s), ) that has the mode of each column. Assumes labels of 1 or 2.
    '''
    
    predictions = np.zeros((len(s[0]), )).astype(int)

    for c in range(0, len(s[0])):
        count = 0
        for r in range(0, len(s)):
            if s[r][c] == 1:
                count += 1
        if count >= (len(s) + 1) / 2:
            predictions[c] = 1
        else:
            predictions[c] = 2
            
    return predictions

In [227]:
combined_pop, labels = get_sequences()
#print(combined_pop)

#printmatrix(compare(3, combined_pop))
scores = []

# Run the analysis for every generation
for gen in range(0, len(combined_pop)):
    # Get the alignment matrix for the generation
    print(f'\nScores for generation {gen + 1}:')
    newscores, median = compare(gen, combined_pop)
    printmatrix(newscores, 6)

    # Find the two most similar sequences, and assume the next closest 4 in that row are all in the same group
    predictions = []
    bestcol = np.unravel_index(newscores.argmax(), newscores.shape)[0]
    closest = np.argpartition(newscores[bestcol], -4)[-4:]
    closest = np.append(closest, bestcol)
    #print(closest)
    
    print(f'\nPredictions for generation {gen + 1}:')
    for p in closest:
        #print(f'Key generated from individual {p + 1}')
        temp = copy.deepcopy(newscores)
        predictions.append(predict(temp, median, p))
    
    #printmatrix(predictions)
    final = col_mode(predictions)
    
    print(f'Predicted labels: {final}')
    #print(labels[gen])
    
    errors = 0
    for i in range(0, len(final)):
        if final[i] != labels[gen][i]:
            print(f'Individual {i + 1} was from population {labels[gen][i]}, but {final[i]} was predicted!')
            errors += 1
        
    print(f'Missed {errors} out of {len(final)} predictions.')
    
    scores.append(newscores)



Scores for generation 1:
     0     1     2     3     4     5     6     7     8     9    10    11    12    13    14    15    16    17    18    19    20
     1     0  1334   598  1492  1514  1820   746  1602   720  1510   552   490   470   468   408   502   478   498   556   546
     2  1334     0   720  1312  1470  1314   888  1368   864  1322   584   604   602   586   506   578   614   546   620   542
     3   598   720     0   570   710   558  1392   676  1346   618   668   688   614   546   640   658   564   686   590   638
     4  1492  1312   570     0  1504  1392   766  1596   722  1670   594   582   546   624   468   632   600   550   624   592
     5  1514  1470   710  1504     0  1414   828  1548   810  1556   580   544   546   550   460   526   584   542   624   588
     6  1820  1314   558  1392  1414     0   736  1502   700  1410   592   470   464   494   418   528   442   518   544   562
     7   746   888  1392   766   828   736     0   816  1820   742   594   612   560 

     0     1     2     3     4     5     6     7     8     9    10    11    12    13    14    15    16    17    18    19    20
     1     0  1218  1814  1258  1304  1344  1808  1316  1390  1418   418   450   476   522   542   450   436   476   522   440
     2  1218     0  1172  1374  1604  1682  1180  1682  1432  1546   434   424   466   466   488   432   444   440   474   404
     3  1814  1172     0  1206  1218  1298  1742  1270  1324  1372   408   422   456   488   480   422   414   456   490   418
     4  1258  1374  1206     0  1348  1406  1166  1426  1742  1500   434   452   442   456   490   434   472   458   464   434
     5  1304  1604  1218  1348     0  1644  1264  1722  1412  1540   422   434   462   494   532   448   434   508   474   436
     6  1344  1682  1298  1406  1644     0  1304  1722  1464  1586   446   444   470   474   508   426   452   500   482   442
     7  1808  1180  1742  1166  1264  1304     0  1276  1284  1332   432   424   490   494   514   436   430   